In [2]:
from py_ecc.bn128 import add, multiply, neg, curve_order, G1, Z1

# BASIC OPERATIONS

In [4]:
# Generator point
print("GENERATOR POINT:", G1, "\n\n")

# Add points
print("POINT SUM:", add(G1, G1), "\n\n")

# Multiply point
print("POINT MULTIPLICATION:", multiply(G1, 2), "\n\n")

GENERATOR POINT: (1, 2) 


POINT SUM: (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764) 


POINT MULTIPLICATION: (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764) 




# POINT AT INFINITY AND WRAP AROUND

In [25]:
# Adding a point and its inverse gets you the point at infinity
assert add(G1, neg(G1)) is None

# If you add G1 to itself curve_order times, you get G1
assert add(G1, multiply(G1, curve_order)) == G1

(1368015179489954701390400359078579693043519447331113978918064868415326638035, 11970132820537103637166003141937572314130795164147247315533067598634108082819)


# Exercise 1
### Get inverse without using neg function

In [6]:
field_element = 5
GN = multiply(G1, field_element)

# METHOD 1:
y_val = -GN[1]
GN_inv = (GN[0], y_val)
assert neg(GN) == GN_inv

# METHOD 2:
inverse_field_element = curve_order - field_element
GN_inv = multiply(G1, inverse_field_element)
assert neg(GN) == GN_inv

# Exercise 2
### ZK Addition: I know two numbers that sum to 16.

In [ ]:
"""PROVER:""" 
A = multiply(G1, 5)
B = multiply(G1, 11)
proof = (A, B, 16)

"""VERIFIER:"""
v_point = multiply(G1, 16)
assert add(A, B) == v_point

# Exercise 3
### ZK Multiplication by Constant: "I know that 5 times a hidden number, H, is 55."

In [ ]:

# Prover will:
hidden_number = 11
H = multiply(G1, hidden_number)
proof = (5, H, 55)

"""VERIFIER:"""
P = multiply(G1, 55)
assert multiply(H, 5) == P

# Exercise 4
### ZK Equation: "I know x and y such that 5x + 2y = 16"

In [ ]:
"""PROVER:""" 
x = 2
y = 3

X = multiply(G1, x)
Y = multiply(G1, y)
proof = (5, X, 2, Y, 16)

"""VERIFIER:"""

construct = add(multiply(X, 5), multiply(Y, 2))
assert construct == multiply(G1, 16)

# Exercise 5
### ZK Rational Number Math: how to do the following on Elliptic Curves?

$$  5/2 + 1/2 = 3 $$

$$ 5 * (1/2) + (1) * (1/2) = 3 $$

In [5]:
inv_field_elem = pow(2, -1, curve_order)

inv2_point = multiply(G1, inv_field_elem)

checkpoint = multiply(G1, 3)

assert add(multiply(inv2_point, 5), inv2_point) == checkpoint
